In [ ]:
import pandas as pd # type: ignore
from scipy.stats import ttest_ind # type: ignore
from sklearn.feature_extraction.text import CountVectorizer # type: ignore
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # type: ignore
from textblob import TextBlob # type: ignore
from wordcloud import WordCloud # type: ignore
import glob # type: ignore
import os # type: ignore
import swifter

In [ ]:
# Definir la ruta de la carpeta
folder_path = os.path.join("Google Maps", "reviews-estados", "review-Florida")

# Lista para almacenar los DataFrames de cada archivo
df_list = []

# Iterar sobre los archivos del 1 al 19
for i in range(1, 20):  # del 1 al 19
    file_path = os.path.join(folder_path, f"{i}.json")
    
    if os.path.exists(file_path):  # Verificar si el archivo existe
        try:
            df_temp = pd.read_json(file_path, lines=True)  # Cargar el JSON línea por línea
            df_list.append(df_temp)  # Agregar al listado de DataFrames
        except ValueError as e:
            print(f"Error al procesar {file_path}: {e}")

# Concatenar todos los DataFrames en uno solo
reviews_gm = pd.concat(df_list, ignore_index=True)

In [ ]:
gmap_id = pd.read_csv('metadatos_fl_final.csv', usecols= ['id','gmap_id'])

In [ ]:
# Filtrar reviews_gm para que solo contenga las filas con gmap_id presentes en gmap_id
filtered_reviews_gm = reviews_gm[reviews_gm['gmap_id'].isin(gmap_id['gmap_id'])]

In [ ]:
reviews_gm = filtered_reviews_gm.copy()

In [ ]:
# Corregir el formato de time (convertir de milisegundos a segundos)
reviews_gm["time"] = reviews_gm["time"] // 1000  # División entera para evitar decimales

# Convertir 'time' de timestamp UNIX a datetime
reviews_gm["date"] = pd.to_datetime(reviews_gm["time"], unit="s")

# Cambiar el formato
reviews_gm['date'] = pd.to_datetime(reviews_gm['date']).dt.strftime('%Y-%m-%d')
reviews_gm['date'] = pd.to_datetime(reviews_gm['date'])  # Mantiene datetime sin la hora

reviews_gm = reviews_gm.drop(columns="time")

# Definir el rango de fechas permitido
end_date = pd.Timestamp("2022-01-19")
start_date = end_date - pd.DateOffset(years=5)  # Calcula la fecha mínima automáticamente

# Filtrar solo los registros dentro del rango de los últimos 5 años
reviews_gm = reviews_gm[(reviews_gm["date"] >= start_date) & (reviews_gm["date"] <= end_date)]

In [ ]:
# Eliminar las columnas 'pics' y 'resp' porque tienen muchos nulos
reviews = reviews_gm.drop(columns=["pics", "resp"])

# Eliminar las columnas 'name' porque no es util para los análisis
reviews = reviews.drop(columns=["name"])

In [ ]:
# Definir el tamaño del chunk
chunk_size = 50000  # Procesar en bloques de 50000 reviews

# Función de análisis de sentimiento
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    vader_score = analyzer.polarity_scores(text)['compound']
    textblob_score = TextBlob(text).sentiment.polarity
    return pd.Series([vader_score, textblob_score])

# Iterar sobre los datos en chunks
for start in range(0, len(reviews), chunk_size):
    end = min(start + chunk_size, len(reviews))
    chunk = reviews.iloc[start:end].copy()  # Copia para evitar modificaciones en el DataFrame original

    # Aplicar análisis de sentimiento con swifter
    chunk[['vader_score', 'textblob_score']] = chunk['text'].swifter.apply(lambda x: analyze_sentiment(str(x))).apply(pd.Series)

    # 🔹 Guardar cada chunk en un archivo Parquet para no perder datos
    chunk.to_parquet(f"sentiment_chunk_gm_{start}.parquet")

    print(f"✅ Procesado y guardado chunk {start}-{end}")

In [ ]:
# Buscar todos los archivos guardados
chunk_files = glob.glob("sentiment_chunk_gm_*.parquet")

# Cargar y combinar todos los chunks
reviews_final = pd.concat([pd.read_parquet(f) for f in chunk_files], ignore_index=True)

# Verificar que todo esté unido
print(f"Dataset final: {reviews_final.shape[0]} filas, {reviews_final.shape[1]} columnas")


In [ ]:
reviews_final.info()

In [ ]:
gmap_id.info()

In [ ]:
reviews_final['has_text'] = reviews_final['text'].notnull().astype(int)

In [ ]:
reviews_final = reviews_final.drop(columns=['text', 'user_id'])

In [ ]:
# Unir reviews_final con el DataFrame de gmap_id para agregar la columna 'id'
reviews_final = reviews_final.merge(gmap_id, on="gmap_id", how="left")

# Verificar la estructura después del merge
print(reviews_final.info())
print(reviews_final.head())


In [ ]:
# Definir el orden lógico de las columnas
column_order = ['id','gmap_id', 'rating', 'has_text', 'vader_score', 'textblob_score','date']

# Aplicar el orden de columnas asegurando que no haya errores por columnas faltantes
reviews_final = reviews_final[column_order]

# Reiniciar el índice después de todas las transformaciones
reviews_final.reset_index(drop=True, inplace=True)

In [ ]:
reviews_final.to_csv('reviews_gm.csv', index = False)